In [1]:
import pickle
import argparse
import shutil
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from unimol_tools import MolTrain, MolPredict
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from unimol_tools.utils import YamlHandler, logger, Logger
import random
from rdkit import Chem
from rdkit.Chem import AllChem

def result_plot(target, predict, nmrtype, mode, fig_title, fig_path_train):
    mae = mean_absolute_error(target, predict)
    rmse = np.sqrt(mean_squared_error(target, predict))
    R2 = r2_score(target, predict)

    fig, ax = plt.subplots(figsize=(5, 5), dpi=150)

    ax.scatter(target, predict, alpha=0.1, s=10, label=f'{nmrtype} NMR_{mode}')

    ax.text(0.95, 0.11, "MAE = {:>7.4f}".format(mae), fontsize=12, ha='right', transform=ax.transAxes)
    ax.text(0.95, 0.07, "RMSE = {:>7.4f}".format(rmse), fontsize=12, ha='right', transform=ax.transAxes)
    ax.text(0.95, 0.02, "R$^2$ = {:>7.4f}".format(R2), fontsize=12, ha='right', transform=ax.transAxes)
    ax.set_xlabel('target')
    ax.set_ylabel('predict')

    x_min, x_max = ax.get_xlim()
    y_min, y_max = ax.get_ylim()

    ax.set_xlim(min(x_min, y_min), max(x_max, y_max))  # 设置 x 轴范围
    ax.set_ylim(min(x_min, y_min), max(x_max, y_max))  # 设置 y 轴范围

    _ = ax.plot([min(x_min, y_min), max(x_max, y_max)], [min(x_min, y_min), max(x_max, y_max)], c='k', ls='--')


    plt.title(fig_title,y=1.02)
    ax.set_aspect('equal') 
    ax.legend(loc='upper left')

    plt.tight_layout()
    plt.savefig(fig_path_train)
    plt.show()

def random_split(dataset, random_ratio=0.8):
    if random_ratio == None:
        return dataset, dataset
    else:
        key1 = list(dataset.keys())[0]
        list_length = len(dataset[key1])
        indices = list(range(list_length))
        random.shuffle(indices)
        train_split_index = int(random_ratio * list_length)
        random_set1 = {}
        random_set2 = {}
        for key in dataset:
            random_set1[key] = [dataset[key][i] for i in indices[:train_split_index]]
            random_set2[key] = [dataset[key][i] for i in indices[train_split_index:]]

        return random_set1, random_set2


# 从总数据库获得筛选指定核磁谱
def filter_data(molecules_nmrdata, filter_list, datatype, sampling_ratio=None):
    filtered_indices = []
    # print("sampling_ratio1", sampling_ratio)
    if 'all' in filter_list:
        if sampling_ratio==None:
            return molecules_nmrdata
        else:
            molecules_nmrdata, molecules_nmrdata2 = random_split(molecules_nmrdata, sampling_ratio)
            return molecules_nmrdata

    for i, atom_mask in enumerate(molecules_nmrdata['atom_mask']):
        if all(x == 0 for x in atom_mask):
            continue
        if any(x in filter_list for x in atom_mask):
            filtered_indices.append(i)
            
    filtered_dict = {}
    for key in molecules_nmrdata.keys():
        filtered_dict[key] = [molecules_nmrdata[key][i] for i in filtered_indices]

    filtered_dict, filtered_dict2 = random_split(filtered_dict, sampling_ratio)

    return filtered_dict

# 获得训练的元素列表
def map_element(nmrtype):
    result = []
    if nmrtype == 'ALL':
        result.append('all')
        return result
    for element in nmrtype.split("+"):
        result.append(Chem.GetPeriodicTable().GetAtomicNumber(element))

    return result

/opt/conda/envs/uninmr/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


fused_multi_tensor is not installed corrected
fused_layer_norm is not installed corrected
fused_softmax is not installed corrected


In [2]:
import numpy as np

import numpy as np
from rdkit import Chem

def parse_xyz_file(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    natoms = int(lines[0])
    comment = lines[1].strip()

    elements = []
    coords = []
    atomic_numbers = []

    periodic_table = Chem.GetPeriodicTable()

    for line in lines[2:2+natoms]:
        parts = line.split()
        element = parts[0]
        x, y, z = map(float, parts[1:4])
        elements.append(element)
        coords.append([x, y, z])
        atomic_number = periodic_table.GetAtomicNumber(element)
        atomic_numbers.append(atomic_number)

    coords = np.array(coords)
    return elements, coords, atomic_numbers


In [3]:
import os

def process_all_xyz_files(folder_path):
    infer_data = {
        'atoms': [],
        'coordinates': [],
        'atom_target': [],
        'atom_mask': [],
    }

    all_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.xyz'):
                filepath = os.path.join(root, file)
                all_files.append(filepath)

    # 按照文件名进行排序
    all_files.sort()

    for filepath in all_files:
        print(f"Processing {filepath}")
        elements, coords, atomic_numbers = parse_xyz_file(filepath)
        infer_data['atoms'].append(elements)
        infer_data['coordinates'].append(coords)
        infer_data['atom_target'].append([0] * 512)
        infer_data['atom_mask'].append([0] + atomic_numbers + (512 - 1 - len(atomic_numbers)) * [0])

    return infer_data

def calculate_mad(arr, ref):
    return np.mean(np.abs(arr - ref))

# 定义一个函数来计算RMSD
def calculate_rmsd(arr, ref):
    return np.sqrt(np.mean((arr - ref) ** 2))

In [9]:
def uq(data):
    data = np.array(data)
    mean = np.mean(data)
    variance = np.var(data)
    std = np.std(data)
    return mean, variance, std


In [4]:
infer_data = process_all_xyz_files('/vepfs/fs_users/xufanjie/data/NMR/isomer/isomer_identification/example/hyacinhacines')


Processing /vepfs/fs_users/xufanjie/data/NMR/isomer/isomer_identification/example/hyacinhacines/xyz/isomer_1.xyz
Processing /vepfs/fs_users/xufanjie/data/NMR/isomer/isomer_identification/example/hyacinhacines/xyz/isomer_2.xyz
Processing /vepfs/fs_users/xufanjie/data/NMR/isomer/isomer_identification/example/hyacinhacines/xyz/isomer_3.xyz
Processing /vepfs/fs_users/xufanjie/data/NMR/isomer/isomer_identification/example/hyacinhacines/xyz/isomer_4.xyz
Processing /vepfs/fs_users/xufanjie/data/NMR/isomer/isomer_identification/example/hyacinhacines/xyz/isomer_5.xyz
Processing /vepfs/fs_users/xufanjie/data/NMR/isomer/isomer_identification/example/hyacinhacines/xyz/isomer_6.xyz
Processing /vepfs/fs_users/xufanjie/data/NMR/isomer/isomer_identification/example/hyacinhacines/xyz/isomer_7.xyz
Processing /vepfs/fs_users/xufanjie/data/NMR/isomer/isomer_identification/example/hyacinhacines/xyz/isomer_8.xyz


In [5]:

nmrtype = 'ALL'
datatype='points3d'
# nmr_data_file_path_test = '/vepfs/fs_users/xufanjie/data/NMR/dataset/nmrshiftdb2_2024/mol_nmrshift_nmrshiftdb2_2024_test.pkl'
filter_list = map_element(nmrtype)

filtered_data_test = infer_data
filtered_data_test = filter_data(filtered_data_test, filter_list, datatype)

# 模型预测
logger.info("Start Evaluation on UniNMR!")
folder_path='/vepfs/fs_ckps/xufanjie/UniNMR/results2024old/weight_pretrain_self_supervised_nmrshiftdb2_2024_ALL_sampling_ratio_1.0_batchsize_8_gradaccum_1_epochs_50_optimizer_adam_lr_0.0001_loss_l1_metric_mae_onlyatom_earlystopping_10'
# folder_path='/vepfs/fs_ckps/xufanjie/UniNMR/resultdft/weight_small_molecule_pretrain_self_supervised_ALL_sampling_ratio_None_batchsize_8_gradaccum_1_epochs_50_optimizer_adam_lr_0.0001_loss_l1_metric_mae_onlyatom_earlystopping_10'
clf = MolPredict(load_model=folder_path)
test_pred = clf.predict(filtered_data_test, datatype = datatype)
print(f" test_predict {test_pred.shape}")
# 画图
target = clf.cv_true[clf.cv_label_mask]
predict = clf.cv_pred[clf.cv_label_mask]
index_mask = np.array(clf.cv_index_mask.tolist()).astype(int)

data_dict = {
    'cv_true': clf.cv_true,
    'cv_pred': clf.cv_pred,
    'cv_pred_fold': clf.cv_pred_fold,
    'cv_label_mask': clf.cv_label_mask,
    'index_mask': index_mask
}

2024-11-25 13:44:53 | None | 10 | INFO | UniNMR | Start Evaluation on UniNMR!
2024-11-25 13:44:53 | unimol_tools/models/unimol.py | 151 | INFO | UniNMR | Loading pretrained weights from /vepfs/fs_ckps/xufanjie/UniNMR/saved/mol_pre_all_h_220816.pt
2024-11-25 13:44:53 | unimol_tools/models/nnmodel.py | 102 | INFO | UniNMR | Model structure:
UniMolModel(
  (embed_tokens): Embedding(31, 512, padding_idx=0)
  (encoder): TransformerEncoderWithPair(
    (emb_layer_norm): LayerNorm(torch.Size([512]), eps=1e-05, elementwise_affine=True)
    (final_layer_norm): LayerNorm(torch.Size([512]), eps=1e-05, elementwise_affine=True)
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): SelfMultiheadAttention(
          (in_proj): Linear(in_features=512, out_features=1536, bias=True)
          (out_proj): Linear(in_features=512, out_features=512, bias=True)
        )
        (self_attn_layer_norm): LayerNorm(torch.Size([512]), eps=1e-05, elementwise_affine=True)
        (fc1)

 test_predict (8, 512)


In [10]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [13]:
# mp = clf.cv_pred
for fold in range(5):
    mp = (clf.cv_pred_fold)[fold]
    mm = index_mask
    target = [
        [39.04, 28.81, 18.52, 28.56, 24.53, 43.66, 32.64, 40.08, 28.07, 20.62, 37.63, 75.64, 24.27, 24.29, 26.81],
        [75.64, 32.64, 28.56, 28.07, 43.66, 37.63, 24.29, 39.04, 24.53, 28.81, 20.62, 18.52, 40.08, 26.81, 24.27],
    ]

    data = {
        'predict': [],
        'target': [],
        'maes': [],
        'rmses': [],
    }

    for i in range(2):
        if i % 2 == 0:
            maes = []
            rmses = []
        predict = mp[i][mm[i]==6]
        mae = mean_absolute_error(target[i], predict)
        rmse = np.sqrt(mean_squared_error(target[i], predict))

        data['predict'].append(predict)
        data['target'].append(target[i])
        data['maes'].append(mae)
        data['rmses'].append(rmse)


        maes.append(mae)
        rmses.append(rmse)
        
        if i % 2 == 1:
            if maes[0]>=maes[1]:
                minmae = 'b'
            else:
                minmae = 'a'
            if rmses[0]>=rmses[1]:
                minrmse = 'b'
            else:
                minrmse = 'a'
            print("i", int(i/2), "mae", maes, "minmae", minmae, "rmse", rmses, "minrmse", minrmse, '\n')

i 0 mae [4.8672620035807315, 1.2288107706705726] minmae b rmse [6.699835445198132, 1.520002508932294] minrmse b 

i 0 mae [4.192194691975912, 1.1742703755696615] minmae b rmse [6.2016396509775, 1.5894811027385254] minrmse b 

i 0 mae [4.60225912475586, 1.7060412597656254] minmae b rmse [6.184950499685545, 2.0652568961167126] minrmse b 

i 0 mae [5.169836741129558, 1.1819454040527342] minmae b rmse [6.977980713960354, 1.6375676016523333] minrmse b 

i 0 mae [4.747939727783203, 1.2299903971354165] minmae b rmse [5.789124148450424, 1.566385691946951] minrmse b 



In [ ]:

r1 = [6.699835445198132, 6.2016396509775, 6.184950499685545, 6.977980713960354, 5.789124148450424]
r2 = [1.520002508932294, 1.5894811027385254, 2.0652568961167126, 1.6375676016523333, 1.566385691946951]
uq(r1), uq(r2)

((6.37070609165439, 0.17568695124654443, 0.419150272869462),
 (1.6757387602773632, 0.03936673450378338, 0.1984105201439263))

In [19]:
# tic10 C
# mp = clf.cv_pred
for fold in range(5):
    mp = (clf.cv_pred_fold)[fold]
    mm = index_mask

    predict = mp[mm==6].reshape(mm.shape[0],-1)

    new_order1 = [10,26,27,29,13,14,15,24,12,17,18,19,20,21,8,7,6,5,4,3,2,1,22,23]
    new_order2 = [15,13,12,10,18,19,20,29,17,22,23,24,25,26,8,7,6,5,4,3,2,1,28,27]
    new_order3 = [17,11,10,13,15,29,28,19,16,21,22,23,24,25,8,7,6,5,4,3,2,1,27,26]
    new_order1 = np.argsort(np.argsort(new_order1)).tolist()
    new_order2 = np.argsort(np.argsort(new_order2)).tolist()
    new_order3 = np.argsort(np.argsort(new_order3)).tolist()
    # new_order = [i - 1 for i in new_order]
    predict[0,:] = predict[0, new_order1]
    predict[1,:] = predict[1, new_order2]
    predict[2,:] = predict[2, new_order3]


    exp = np.array([159.39, 49.38, 52.67, 156.98, 150.05, 29.10, 51.07, 50.22, 105.22, 65.47, 140.27, 128.35, 127.64, 126.01, 43.47, 136.22, 127.97, 124.58, 126.07, 128.69, 135.54, 21.07])
    c_rmsds = []
    for i in range(3):
        mad = calculate_mad(predict[i, :-2], exp)
        rmsd = calculate_rmsd(predict[i, :-2], exp)
        c_rmsds.append(rmsd)
    print(c_rmsds)

[2.5398384140681434, 2.9518762964293668, 3.043956495785452]
[2.425174680980296, 2.514499743564811, 2.9779345338092984]
[3.01664151833165, 3.2115081962259513, 3.298839221359835]
[2.7682439288759277, 2.983260643453268, 3.3463201815605745]
[2.4535616622639784, 2.9113778374010137, 3.142911302903656]


In [21]:
r1 = [2.5398384140681434,2.425174680980296, 3.01664151833165, 2.7682439288759277, 2.4535616622639784]
r2 = [2.9518762964293668,  2.514499743564811,  3.2115081962259513, 2.983260643453268, 2.9113778374010137]
r3 = [3.043956495785452, 2.9779345338092984, 3.298839221359835,  3.3463201815605745, 3.142911302903656]
uq(r1), uq(r2), uq(r3)

((2.640692040903999, 0.04984889175754276, 0.22326865377285446),
 (2.9145045434148824, 0.050869767038515, 0.2255432708783727),
 (3.161992347083763, 0.020175528925627954, 0.1420405890076071))

In [39]:
# nevirapine C
# mp = clf.cv_pred
UQ_list = []
for fold in range(5):
    mp = (clf.cv_pred_fold)[fold]
    mm = index_mask

    predict = mp[mm==6].reshape(mm.shape[0],-1)

    new_order1 = [6, 20, 2, 3, 4, 1, 14, 8, 13, 12, 11, 10, 17, 18, 19]
    new_order2 = [6, 11, 2, 3, 4, 1, 13, 20, 15, 16, 17, 18, 8, 9, 10]
    new_order3 = [6, 8, 2, 3, 4, 1, 10, 17, 12, 13, 14, 15, 19, 20, 21]
    new_order4 = [6, 11, 2, 3, 4, 1, 19, 13, 18, 17, 16, 15, 8, 9, 10]
    new_order1 = np.argsort(np.argsort(new_order1)).tolist()
    new_order2 = np.argsort(np.argsort(new_order2)).tolist()
    new_order3 = np.argsort(np.argsort(new_order3)).tolist()
    new_order4 = np.argsort(np.argsort(new_order4)).tolist()
    # new_order = [i - 1 for i in new_order]
    predict[0,:] = predict[0, new_order1]
    predict[1,:] = predict[1, new_order2]
    predict[2,:] = predict[2, new_order3]
    predict[3,:] = predict[3, new_order4]


    exp = np.array([156.8, 119.5, 143.6, 114.4, 144.2, 17.5, 169.2, 157.6, 109.7, 139.8, 112.7, 151.1, 24.3, 6.6])
    c_rmsds = []
    c_mads = []
    for i in range(4):
        mad = calculate_mad(predict[i, :-1], exp)
        rmsd = calculate_rmsd(predict[i, :-1], exp)
        c_mads.append(mad)
        c_rmsds.append(rmsd)
    UQ_list.append(c_rmsds)


In [40]:
uq_data = np.array(UQ_list)

In [43]:
uq(uq_data[:,0]), uq(uq_data[:,1]), uq(uq_data[:,2]), uq(uq_data[:,3])


((7.948185040573371, 0.13842236320884543, 0.3720515598796025),
 (5.203607525995681, 0.03563843044292002, 0.1887814356416436),
 (4.891039476542903, 0.1376705425762959, 0.37103981265666885),
 (2.6643646336013553, 0.04880013832369959, 0.2209075334245068))

In [47]:

targets =  [
        [14.6, 16.6, 21.3, 24.3, 25.2, 33.4, 39.4, 42.3, 78.20, 181.4],
        [9.9, 15.7, 21.3, 24.3, 25.3, 33.5, 41.6, 41.8, 76.3, 181.9],
        [11.1, 14.5, 21.6, 23.6, 25, 33.3, 42.1, 42.6, 75.6, 181.6],
        [12.5, 14.2, 22.2, 23.1, 25, 32.1, 43.1, 43.2, 75.6, 181.4],
    ]
# mp = clf.cv_pred

UQ_list = []
for fold in range(5):
    mp = (clf.cv_pred_fold)[fold]

    mm = index_mask
    data = {
        'predict': [],
        'target': [],
        'maes': [],
        'rmses': [],
    }

    for exp in range(len(targets)):

        target = targets[exp]
        maes = []
        rmses = []
        predicts = []
        for j in range(mm.shape[0]):
            predict = mp[j][mm[j]==6]
            mae = mean_absolute_error(target, predict)
            rmse = np.sqrt(mean_squared_error(target, predict))

            predicts.append(predict)
            maes.append(mae)
            rmses.append(rmse)
        UQ_list.append(rmses)
        data['predict'].append(predicts)
        data['target'].append(target)
        data['maes'].append(maes)
        data['rmses'].append(rmses)

In [51]:
uq_data = np.array(UQ_list).reshape(5,4,4)

In [57]:
exp_index = 0
uq(uq_data[:, exp_index,0]), uq(uq_data[:, exp_index,1]), uq(uq_data[:, exp_index,2]), uq(uq_data[exp_index, :,3])


((1.3481485087146468, 0.014165447548379187, 0.11901868571102264),
 (1.7423077990274425, 0.013202825173865604, 0.11490354726406668),
 (1.7395372360839185, 0.0009721418764648364, 0.031179189798082253),
 (1.2509762410872547, 0.03338084752004246, 0.18270426245723567))

In [58]:
exp_index = 1
uq(uq_data[:, exp_index,0]), uq(uq_data[:, exp_index,1]), uq(uq_data[:, exp_index,2]), uq(uq_data[exp_index, :,3])


((1.6487512704812137, 0.016947315568276902, 0.13018185575677166),
 (1.3171455930465825, 0.0540380675061814, 0.2324608945740797),
 (1.3762679566146068, 0.07218885751999221, 0.2686798420425176),
 (1.4835359034878097, 0.0521444719173428, 0.22835164093420218))

In [59]:
exp_index = 2
uq(uq_data[:, exp_index,0]), uq(uq_data[:, exp_index,1]), uq(uq_data[:, exp_index,2]), uq(uq_data[exp_index, :,3])


((1.4522843196575288, 0.024060916963056247, 0.15511581790087126),
 (1.1960278142954757, 0.028318200197343452, 0.16828012418982657),
 (1.1999194346755853, 0.034957045699267955, 0.18696803389688826),
 (1.3956836421627994, 0.003624743737858779, 0.06020584471510037))

In [60]:
exp_index = 3
uq(uq_data[:, exp_index,0]), uq(uq_data[:, exp_index,1]), uq(uq_data[:, exp_index,2]), uq(uq_data[exp_index, :,3])


((1.5394530011115488, 0.0199878117155729, 0.14137825757722755),
 (1.522382545173411, 0.03743223976669126, 0.19347413203498617),
 (1.394581982519206, 0.05968243837002453, 0.24429989433076824),
 (1.3190758026190414, 0.012148896309291257, 0.11022203186882039))

In [85]:
# mp = clf.cv_pred
UQ_list = []

for fold in range(5):
    mp = (clf.cv_pred_fold)[fold]

    mm = index_mask

    predict = mp[mm==6].reshape(mm.shape[0],-1)
    exp1 = np.array([14.3, 19.6, 30.9, 83.6, 73.1, 44.3, 78.7, 143, 125.9, 128.4, 127.3, 128.4, 125.9])
    exp2 = np.array([14.1, 19.1, 35.3, 85.6, 76.8, 43.4, 79.7, 143.3, 125.6, 128.5, 127.3, 128.5, 125.6])
    exp3 = np.array([14.1, 19.1, 35.6, 87.1, 76.7, 44, 79.5, 142.1, 125.8, 128.3, 127.4, 128.3, 125.8])
    exp4 = np.array([14.3, 19.7, 31.3, 83.1, 73.8, 44.7, 78.2, 143.8, 125.3, 128.3, 127.1, 128.3, 125.3])


    exps = (exp1, exp2, exp3, exp4)

    mad_pick = []
    rmsd_pick = []
    rmsds = []
    for index, exp_index in enumerate([1,2,3,4]):
        
        exp = exps[index]
        mad_values = np.apply_along_axis(calculate_mad, 1, predict, exp)
        rmsd_values = np.apply_along_axis(calculate_rmsd, 1, predict, exp)
        mad_pick.append(np.argmin(mad_values)+1)
        rmsd_pick.append(np.argmin(rmsd_values)+1)
        print(f"result of exp{exp_index}:")
        row_rmsd = []
        for i, (mad, rmsd) in enumerate(zip(mad_values, rmsd_values)):
            print(f"Row {i+1}: MAD = {mad:.2f}, RMSD = {rmsd:.2f}")
            row_rmsd.append(rmsd)
        # break
        rmsds.append(row_rmsd)
    # print(mad_pick==[1,2,3,4],mad_pick)
    # print(rmsd_pick==[1,2,3,4],rmsd_pick)
    c_rmsds = np.array(rmsds)
    UQ_list.append(c_rmsds)
    

result of exp1:
Row 1: MAD = 0.77, RMSD = 0.99
Row 2: MAD = 1.02, RMSD = 1.51
Row 3: MAD = 0.99, RMSD = 1.40
Row 4: MAD = 0.66, RMSD = 0.96
result of exp2:
Row 1: MAD = 0.70, RMSD = 1.14
Row 2: MAD = 0.74, RMSD = 1.02
Row 3: MAD = 0.58, RMSD = 0.77
Row 4: MAD = 0.66, RMSD = 0.95
result of exp3:
Row 1: MAD = 0.81, RMSD = 1.28
Row 2: MAD = 0.60, RMSD = 0.93
Row 3: MAD = 0.52, RMSD = 0.76
Row 4: MAD = 0.71, RMSD = 1.17
result of exp4:
Row 1: MAD = 0.79, RMSD = 1.14
Row 2: MAD = 1.11, RMSD = 1.61
Row 3: MAD = 1.03, RMSD = 1.45
Row 4: MAD = 0.77, RMSD = 1.02
result of exp1:
Row 1: MAD = 0.55, RMSD = 0.71
Row 2: MAD = 1.04, RMSD = 1.44
Row 3: MAD = 1.12, RMSD = 1.67
Row 4: MAD = 0.56, RMSD = 0.71
result of exp2:
Row 1: MAD = 0.79, RMSD = 1.38
Row 2: MAD = 0.61, RMSD = 0.90
Row 3: MAD = 0.73, RMSD = 1.00
Row 4: MAD = 0.81, RMSD = 1.41
result of exp3:
Row 1: MAD = 1.05, RMSD = 1.67
Row 2: MAD = 0.78, RMSD = 1.09
Row 3: MAD = 0.69, RMSD = 1.04
Row 4: MAD = 1.06, RMSD = 1.67
result of exp4:
Row 

In [86]:
uq_data_c = np.array(UQ_list).reshape(5,4,4)
uq_data = uq_data_c

In [69]:
exp_index = 0
uq(uq_data[:, exp_index,0]), uq(uq_data[:, exp_index,1]), uq(uq_data[:, exp_index,2]), uq(uq_data[exp_index, :,3])


((1.0713547713084222, 0.04740375394808781, 0.2177240316273971),
 (1.420043968974909, 0.03857749887113037, 0.1964115548309986),
 (1.5921650459957293, 0.01791799371863697, 0.13385811039543688),
 (1.0240123953745515, 0.007901948792162675, 0.08889290630957385))

In [70]:
exp_index = 1
uq(uq_data[:, exp_index,0]), uq(uq_data[:, exp_index,1]), uq(uq_data[:, exp_index,2]), uq(uq_data[exp_index, :,3])


((1.3791821894591167, 0.06240190900665128, 0.2498037409780952),
 (0.9644553530602081, 0.008968971528408334, 0.09470465420668793),
 (0.9144214570073406, 0.008080401565344144, 0.08989105386713488),
 (1.1525977284519238, 0.16194775321138746, 0.4024273266210776))

In [71]:
exp_index = 2
uq(uq_data[:, exp_index,0]), uq(uq_data[:, exp_index,1]), uq(uq_data[:, exp_index,2]), uq(uq_data[exp_index, :,3])


((1.6459862651053565, 0.09663114216007293, 0.3108555004500852),
 (1.120700759455124, 0.02403225885654929, 0.1550234138978667),
 (1.0163370383644827, 0.022960338009321186, 0.15152669074892774),
 (1.391140665564925, 0.07144853796846735, 0.2672985932781304))

In [72]:
exp_index = 3
uq(uq_data[:, exp_index,0]), uq(uq_data[:, exp_index,1]), uq(uq_data[:, exp_index,2]), uq(uq_data[exp_index, :,3])


((1.195923481344983, 0.047572370403499625, 0.2181109130774974),
 (1.4965827324513077, 0.022913697478991763, 0.15137271048307144),
 (1.6487787655537658, 0.01831695965319578, 0.13534016274999738),
 (1.1783104218643194, 0.02823803207309252, 0.1680417569328901))

In [87]:
mp = clf.cv_pred
UQ_list = []

exp1 = np.array([0.92, 0.92, 0.92, 1.4, 1.47, 1.63, 1.71, 3.68, 4.19, 1.83, 2.62, 4.78, 7.25, 7.24, 7.17, 7.24, 7.25])
exp2 = np.array([0.89, 0.89, 0.89, 1.37, 1.46, 1.46, 1.46, 3.9, 4.05, 1.87, 2.57, 4.96, 7.26, 7.25, 7.18, 7.25, 7.26])
exp3 = np.array([0.91, 0.91, 0.91, 1.41, 1.46, 1.5, 1.54, 3.83, 4.09, 1.89, 2.13, 5.04, 7.27, 7.26, 7.19, 7.26, 7.27])
exp4 = np.array([0.94, 0.94, 0.94, 1.4, 1.48, 1.58, 1.66, 3.99, 4.3, 2.02, 2.39, 5.19, 7.26, 7.25, 7.18, 7.25, 7.26])



for fold in range(5):
    mp = (clf.cv_pred_fold)[fold]
    mm = index_mask

    predict = mp[mm==1].reshape(mm.shape[0],-1)
    for row in predict:
        row[3:5] = np.sort(row[3:5])
        row[5:7] = np.sort(row[5:7])
        row[9:11] = np.sort(row[9:11])
    predict = predict[:, :-1]


    exps = (exp1, exp2, exp3, exp4)

    mad_pick = []
    rmsd_pick = []
    rmsds = []
    for index, exp_index in enumerate([1,2,3,4]):
        
        exp = exps[index]
        mad_values = np.apply_along_axis(calculate_mad, 1, predict, exp)
        rmsd_values = np.apply_along_axis(calculate_rmsd, 1, predict, exp)
        mad_pick.append(np.argmin(mad_values)+1)
        rmsd_pick.append(np.argmin(rmsd_values)+1)
        print(f"result of exp{exp_index}:")
        row_rmsd = []
        for i, (mad, rmsd) in enumerate(zip(mad_values, rmsd_values)):
            print(f"Row {i+1}: MAD = {mad:.2f}, RMSD = {rmsd:.2f}")
            row_rmsd.append(rmsd)
        # break
        rmsds.append(row_rmsd)
    # print(mad_pick==[1,2,3,4],mad_pick)
    # print(rmsd_pick==[1,2,3,4],rmsd_pick)
    h_rmsds = np.array(rmsds)
    UQ_list.append(h_rmsds)


result of exp1:
Row 1: MAD = 0.14, RMSD = 0.18
Row 2: MAD = 0.14, RMSD = 0.19
Row 3: MAD = 0.15, RMSD = 0.20
Row 4: MAD = 0.16, RMSD = 0.20
result of exp2:
Row 1: MAD = 0.13, RMSD = 0.16
Row 2: MAD = 0.13, RMSD = 0.16
Row 3: MAD = 0.12, RMSD = 0.16
Row 4: MAD = 0.13, RMSD = 0.17
result of exp3:
Row 1: MAD = 0.12, RMSD = 0.16
Row 2: MAD = 0.11, RMSD = 0.15
Row 3: MAD = 0.10, RMSD = 0.13
Row 4: MAD = 0.12, RMSD = 0.17
result of exp4:
Row 1: MAD = 0.13, RMSD = 0.15
Row 2: MAD = 0.11, RMSD = 0.14
Row 3: MAD = 0.09, RMSD = 0.12
Row 4: MAD = 0.11, RMSD = 0.13
result of exp1:
Row 1: MAD = 0.14, RMSD = 0.17
Row 2: MAD = 0.15, RMSD = 0.18
Row 3: MAD = 0.15, RMSD = 0.20
Row 4: MAD = 0.17, RMSD = 0.22
result of exp2:
Row 1: MAD = 0.10, RMSD = 0.12
Row 2: MAD = 0.08, RMSD = 0.12
Row 3: MAD = 0.11, RMSD = 0.16
Row 4: MAD = 0.13, RMSD = 0.18
result of exp3:
Row 1: MAD = 0.09, RMSD = 0.11
Row 2: MAD = 0.09, RMSD = 0.12
Row 3: MAD = 0.09, RMSD = 0.13
Row 4: MAD = 0.11, RMSD = 0.15
result of exp4:
Row 

In [88]:
uq_data_h = np.array(UQ_list).reshape(5,4,4)
uq_data = uq_data_h

In [80]:
exp_index = 0
uq(uq_data[:, exp_index,0]), uq(uq_data[:, exp_index,1]), uq(uq_data[:, exp_index,2]), uq(uq_data[exp_index, :,3])


((0.17278400892237838, 0.00018218442710619503, 0.013497571155811516),
 (0.17061163393423945, 0.0006278547270272818, 0.02505702949328355),
 (0.19870340503172781, 5.6424562459503324e-05, 0.007511628482526497),
 (0.16716836728209686, 0.0007535661827359234, 0.027451159952466917))

In [81]:
exp_index = 1
uq(uq_data[:, exp_index,0]), uq(uq_data[:, exp_index,1]), uq(uq_data[:, exp_index,2]), uq(uq_data[exp_index, :,3])


((0.1423361324141079, 0.00033498792995558224, 0.018302675486266542),
 (0.13844087365582083, 0.00034630204663362865, 0.018609192530403586),
 (0.1600453317035276, 0.0003032654792471495, 0.017414519208038718),
 (0.16417105296330617, 0.0017576220737773826, 0.04192400355139502))

In [82]:
exp_index = 2
uq(uq_data[:, exp_index,0]), uq(uq_data[:, exp_index,1]), uq(uq_data[:, exp_index,2]), uq(uq_data[exp_index, :,3])


((0.13415503092589162, 0.0015640870202371677, 0.039548540051905424),
 (0.12634415513666486, 0.0009007458919053617, 0.03001242895710645),
 (0.12380879804449649, 0.0019489064474637948, 0.0441464205509778),
 (0.13047391549419768, 0.0016020306619877888, 0.04002537522607113))

In [84]:
exp_index = 3
uq(uq_data[:, exp_index,0]), uq(uq_data[:, exp_index,1]), uq(uq_data[:, exp_index,2]), uq(uq_data[exp_index, :,3])

((0.12337987340027537, 0.0005405913211023195, 0.023250619800390688),
 (0.14191636952040618, 0.00039175382576268, 0.019792772058574312),
 (0.12112971606538796, 0.0005951596164842732, 0.024395893434844176),
 (0.17326959371223288, 0.00028057665885702546, 0.016750422647116264))

In [89]:
uq_data_all=uq_data_c+uq_data_h*10
uq_data = uq_data_all


In [90]:
exp_index = 0
uq(uq_data[:, exp_index,0]), uq(uq_data[:, exp_index,1]), uq(uq_data[:, exp_index,2]), uq(uq_data[exp_index, :,3])


((2.7991948605322063, 0.0793303409245498, 0.2816564235456912),
 (3.1261603083173037, 0.03968874367693195, 0.1992203395161547),
 (3.5791990963130074, 0.03870241448497509, 0.19672929239179177),
 (2.6956960681955207, 0.06982788982808702, 0.26424967327905446))

In [92]:
exp_index = 1
uq(uq_data[:, exp_index,0]), uq(uq_data[:, exp_index,1]), uq(uq_data[:, exp_index,2]), uq(uq_data[exp_index, :,3])


((2.8025435136001957, 0.02978116822368635, 0.17257221162077732),
 (2.3488640896184165, 0.0677465111275857, 0.260281599671559),
 (2.5148747740426165, 0.029568802419594625, 0.17195581531194176),
 (2.7943082580849854, 0.30225437785427645, 0.5497766617948386))

In [93]:
exp_index = 2
uq(uq_data[:, exp_index,0]), uq(uq_data[:, exp_index,1]), uq(uq_data[:, exp_index,2]), uq(uq_data[exp_index, :,3])


((2.987536574364273, 0.04854219652301168, 0.2203229368972093),
 (2.3841423108217725, 0.033600673701201324, 0.18330486545970712),
 (2.254425018809448, 0.14244068144535943, 0.37741314424031314),
 (2.695879820506902, 0.051537655018794495, 0.22701906311760361))

In [94]:
exp_index = 3
uq(uq_data[:, exp_index,0]), uq(uq_data[:, exp_index,1]), uq(uq_data[:, exp_index,2]), uq(uq_data[exp_index, :,3])


((2.429722215347737, 0.14677967331381833, 0.3831183541855158),
 (2.9157464276553697, 0.05218412788290337, 0.2284384553504584),
 (2.8600759262076454, 0.10144946870459033, 0.31851133214469834),
 (2.911006358986648, 0.06703238793582822, 0.2589061373081531))

In [6]:
mp = clf.cv_pred
UQ_list = []

exp1 = np.array([73.7, 77.5, 71.9, 57.5, 28.8, 26, 67.9, 65.2])
exp2 = np.array([82.9, 79.8, 72.1, 57.7, 27.3, 32.5, 69.2, 65.3])
exp3 = np.array([77.1, 80.8, 70.8, 56.6, 29.5, 25.4, 69.5, 61.7])
exp5 = np.array([79.4, 75.9, 71.3, 56.7, 26.9, 31.7, 69.9, 62.6])
exp6 = np.array([83.7, 81.8, 67.1, 51.2, 28.7, 31.9, 74.8, 59.6])
exp7 = np.array([78.3, 77.6, 66.3, 50.4, 27.6, 26.3, 66.9, 60.8])
exp8 = np.array([76.9, 72.5, 67.1, 49.1, 27.4, 30.9, 72.1, 60.9])

for fold in range(5):
    mp = (clf.cv_pred_fold)[fold]

    mm = index_mask

    predict = mp[mm==6].reshape(mm.shape[0],-1)
    new_order = [7, 8, 2, 3, 4, 5, 6, 1]  
    new_order = [i - 1 for i in new_order]
    predict = predict[:, new_order]

    exps = (exp1, exp2, exp3, exp5, exp6, exp7, exp8)

    mad_pick = []
    rmsd_pick = []
    rmsds = []
    for index, exp_index in enumerate([1,2,3,5,6,7,8]):
        
        exp = exps[index]
        mad_values = np.apply_along_axis(calculate_mad, 1, predict, exp)
        rmsd_values = np.apply_along_axis(calculate_rmsd, 1, predict, exp)
        mad_pick.append(np.argmin(mad_values)+1)
        rmsd_pick.append(np.argmin(rmsd_values)+1)
        print(f"result of exp{exp_index}:")
        row_rmsd = []
        for i, (mad, rmsd) in enumerate(zip(mad_values, rmsd_values)):
            print(f"Row {i+1}: MAD = {mad:.2f}, RMSD = {rmsd:.2f}")
            row_rmsd.append(rmsd)
        # break
        rmsds.append(row_rmsd)
    UQ_list.append(rmsds)

result of exp1:
Row 1: MAD = 2.14, RMSD = 2.41
Row 2: MAD = 3.00, RMSD = 3.85
Row 3: MAD = 2.17, RMSD = 2.87
Row 4: MAD = 2.24, RMSD = 2.88
Row 5: MAD = 2.86, RMSD = 3.32
Row 6: MAD = 3.03, RMSD = 3.38
Row 7: MAD = 2.96, RMSD = 3.35
Row 8: MAD = 3.72, RMSD = 4.28
result of exp2:
Row 1: MAD = 3.16, RMSD = 3.91
Row 2: MAD = 2.11, RMSD = 2.39
Row 3: MAD = 2.57, RMSD = 3.13
Row 4: MAD = 3.51, RMSD = 4.15
Row 5: MAD = 2.36, RMSD = 2.64
Row 6: MAD = 3.20, RMSD = 3.54
Row 7: MAD = 2.77, RMSD = 3.34
Row 8: MAD = 2.70, RMSD = 3.10
result of exp3:
Row 1: MAD = 1.69, RMSD = 2.03
Row 2: MAD = 2.84, RMSD = 3.06
Row 3: MAD = 1.75, RMSD = 2.23
Row 4: MAD = 2.22, RMSD = 2.69
Row 5: MAD = 2.67, RMSD = 2.99
Row 6: MAD = 2.65, RMSD = 3.05
Row 7: MAD = 2.29, RMSD = 2.49
Row 8: MAD = 3.34, RMSD = 3.49
result of exp5:
Row 1: MAD = 2.34, RMSD = 2.96
Row 2: MAD = 1.81, RMSD = 2.02
Row 3: MAD = 1.64, RMSD = 2.36
Row 4: MAD = 2.27, RMSD = 2.94
Row 5: MAD = 1.10, RMSD = 1.33
Row 6: MAD = 1.79, RMSD = 2.34
Row 7:

In [7]:
uq_data_c = np.array(UQ_list)
uq_data = uq_data_c


In [10]:
exp_index = 0
uq(uq_data[:, exp_index,0]), uq(uq_data[:, exp_index,1]), uq(uq_data[:, exp_index,2]), uq(uq_data[:, exp_index,3]), uq(uq_data[:, exp_index,4]), uq(uq_data[:, exp_index,5]), uq(uq_data[:, exp_index,6]), uq(uq_data[:, exp_index,7])


((2.398894104377921, 0.05250193322706368, 0.22913300335626835),
 (3.8866576968666515, 0.18242653911319118, 0.4271141991472435),
 (3.096087988436308, 0.22268907591123752, 0.4718994341077742),
 (2.9952286716143712, 0.009046226310230315, 0.09511165181107052),
 (3.4444841109732165, 0.1185099765188816, 0.3442527799726265),
 (3.9250402824191766, 0.0813294008762827, 0.2851831006148203),
 (3.3412831236986427, 0.025290666791950567, 0.15903039581146294),
 (4.205150821790033, 0.0912367176044627, 0.3020541633622399))

In [11]:
exp_index = 1
uq(uq_data[:, exp_index,0]), uq(uq_data[:, exp_index,1]), uq(uq_data[:, exp_index,2]), uq(uq_data[:, exp_index,3]), uq(uq_data[:, exp_index,4]), uq(uq_data[:, exp_index,5]), uq(uq_data[:, exp_index,6]), uq(uq_data[:, exp_index,7])


((4.165144693751094, 0.16915312150028147, 0.41128228931025157),
 (2.5197388454185723, 0.09544676007071555, 0.30894459061572116),
 (3.521015966126348, 0.1358868231500841, 0.36862829944279113),
 (4.102335726560773, 0.019045898559388787, 0.13800687866693018),
 (3.228306990383129, 0.22832770850232595, 0.47783648720281496),
 (3.029848478814369, 0.10239017142210102, 0.3199846424785118),
 (3.4484109617892864, 0.014598677851920965, 0.12082498852439824),
 (3.2841040588479116, 0.14125229725477434, 0.3758354656691866))

In [12]:
exp_index = 2
uq(uq_data[:, exp_index,0]), uq(uq_data[:, exp_index,1]), uq(uq_data[:, exp_index,2]), uq(uq_data[:, exp_index,3]), uq(uq_data[:, exp_index,4]), uq(uq_data[:, exp_index,5]), uq(uq_data[:, exp_index,6]), uq(uq_data[:, exp_index,7])


((2.3491685910468503, 0.13253256658858031, 0.36405022536537496),
 (2.992821083442394, 0.14487320461930858, 0.3806221283889162),
 (2.4054056708361244, 0.07071296489866938, 0.26591909464848396),
 (2.753157394700614, 0.06175833277874153, 0.2485122386900523),
 (3.250299721810221, 0.07696269959445852, 0.277421519703246),
 (3.1225352274741858, 0.015650024527706623, 0.12510005806436153),
 (2.603516552652464, 0.011381127729130107, 0.10668236840795252),
 (3.509390812745946, 0.06526780227242601, 0.2554756392935068))

In [13]:
exp_index = 3
uq(uq_data[:, exp_index,0]), uq(uq_data[:, exp_index,1]), uq(uq_data[:, exp_index,2]), uq(uq_data[:, exp_index,3]), uq(uq_data[:, exp_index,4]), uq(uq_data[:, exp_index,5]), uq(uq_data[:, exp_index,6]), uq(uq_data[:, exp_index,7])


((2.958776257547422, 0.03374224622716216, 0.18369062639983064),
 (2.3359569715814685, 0.0980181121456624, 0.31307844407697955),
 (2.771140547481653, 0.10949624002304714, 0.3309021608014175),
 (2.708962222672459, 0.02863296018620893, 0.16921276602611557),
 (1.8106686995746817, 0.10997051097766522, 0.3316180196817797),
 (2.4030055492453055, 0.04875841169982191, 0.22081306958561558),
 (2.7117601187084834, 0.15172605648346316, 0.3895202902076645),
 (2.508675068890101, 0.08087922150820519, 0.2843927240774721))

In [14]:
exp_index = 4
uq(uq_data[:, exp_index,0]), uq(uq_data[:, exp_index,1]), uq(uq_data[:, exp_index,2]), uq(uq_data[:, exp_index,3]), uq(uq_data[:, exp_index,4]), uq(uq_data[:, exp_index,5]), uq(uq_data[:, exp_index,6]), uq(uq_data[:, exp_index,7])


((4.779185030714404, 0.17429091162418917, 0.4174816302835242),
 (3.5069934487040357, 0.04107290905010985, 0.20266452341273214),
 (4.391279059207497, 0.1287733369894902, 0.3588500201887833),
 (4.372638717341557, 0.019340706804771686, 0.13907086972033966),
 (4.227722198729213, 0.08384427846203882, 0.28955876512728607),
 (3.2300483881263657, 0.13108563681587884, 0.36205750484678373),
 (3.844454870960574, 0.018952147772956306, 0.13766679982100372),
 (3.3690600766937164, 0.10521192890163214, 0.3243638834729171))

In [15]:
exp_index = 5
uq(uq_data[:, exp_index,0]), uq(uq_data[:, exp_index,1]), uq(uq_data[:, exp_index,2]), uq(uq_data[:, exp_index,3]), uq(uq_data[:, exp_index,4]), uq(uq_data[:, exp_index,5]), uq(uq_data[:, exp_index,6]), uq(uq_data[:, exp_index,7])


((2.8581147553199413, 0.0692302531630266, 0.2631164251106848),
 (3.7694681044300653, 0.3410085938298311, 0.5839594111150458),
 (2.7803338745330155, 0.4769540781069449, 0.6906186198669603),
 (2.590496476428813, 0.08110613579298118, 0.28479138995584324),
 (3.373904501335776, 0.07203170724090503, 0.2683872337517286),
 (3.041268434229628, 0.20406872396603576, 0.45173966392828047),
 (3.157458316609586, 0.0889136499615294, 0.29818391968972674),
 (3.588296212244404, 0.2149596734665411, 0.46363743751614916))

In [16]:
exp_index = 6
uq(uq_data[:, exp_index,0]), uq(uq_data[:, exp_index,1]), uq(uq_data[:, exp_index,2]), uq(uq_data[:, exp_index,3]), uq(uq_data[:, exp_index,4]), uq(uq_data[:, exp_index,5]), uq(uq_data[:, exp_index,6]), uq(uq_data[:, exp_index,7])


((3.4900398012476224, 0.1461849476167568, 0.38234140191294586),
 (4.210253889795764, 0.2943617852564133, 0.5425511821537331),
 (4.083087158782608, 0.13108380004949485, 0.3620549682706962),
 (2.8082442002673913, 0.15483656045726538, 0.39349277052731907),
 (3.1144911134846813, 0.18863210526905624, 0.43431797714238846),
 (3.392515956412891, 0.2583342047964233, 0.5082658800238545),
 (3.648387279491568, 0.2930658467112488, 0.5413555640346267),
 (3.281423445090988, 0.33449662444118633, 0.5783568314122228))

In [18]:
# hyacinhacines H
mp = clf.cv_pred
UQ_list = []

exp1 = np.array([3.87, 3.88, 2.76, 2.65, 3.05, 1.74, 1.93, 1.68, 2.08, 3.47, 3.58, 3.78])
exp2 = np.array([3.76, 3.81, 2.77, 2.81, 2.96, 1.82, 1.90, 1.97, 1.97, 3.32, 3.67, 3.80])
exp3 = np.array([4.05, 4.31, 3.04, 2.66, 3.06, 1.84, 1.97, 1.77, 1.90, 3.72, 3.83, 3.83])
exp5 = np.array([3.89, 4.23, 2.93, 2.70, 2.84, 1.76, 1.83, 1.76, 1.92, 3.37, 3.64, 3.83])
exp6 = np.array([3.96, 4.10, 3.37, 2.92, 3.37, 1.65, 1.92, 1.65, 2.13, 3.01, 3.84, 3.90])
exp7 = np.array([4.15, 3.92, 3.20, 2.91, 3.11, 1.75, 2.00, 1.77, 1.88, 3.82, 3.89, 3.89])
exp8 = np.array([3.76, 3.88, 3.21, 2.79, 2.91, 1.69, 1.87, 1.50, 2.14, 3.39, 3.81, 3.84])

for fold in range(5):
    mp = (clf.cv_pred_fold)[fold]
    mm = index_mask

    predict = mp[mm==1].reshape(mm.shape[0],-1)
    new_order = [12, 14, 4, 5, 6, 7, 8, 9, 10, 11, 2, 3]  
    new_order = [i - 1 for i in new_order]
    predict = predict[:, new_order]
    for row in predict:
        row[3:5] = np.sort(row[3:5])
        row[5:7] = np.sort(row[5:7])
        row[7:9] = np.sort(row[7:9])
        row[10:12] = np.sort(row[10:12])


    exps = (exp1, exp2, exp3, exp5, exp6, exp7, exp8)

    mad_pick = []
    rmsd_pick = []
    rmsds = []
    for index, exp_index in enumerate([1,2,3,5,6,7,8]):
        
        exp = exps[index]
        mad_values = np.apply_along_axis(calculate_mad, 1, predict, exp)
        rmsd_values = np.apply_along_axis(calculate_rmsd, 1, predict, exp)
        mad_pick.append(np.argmin(mad_values)+1)
        rmsd_pick.append(np.argmin(rmsd_values)+1)
        print(f"result of exp{exp_index}:")
        row_rmsd = []
        for i, (mad, rmsd) in enumerate(zip(mad_values, rmsd_values)):
            print(f"Row {i+1}: MAD = {mad:.2f}, RMSD = {rmsd:.2f}")
            row_rmsd.append(rmsd)
        # break
        rmsds.append(row_rmsd)
    UQ_list.append(rmsds)
    
# print(mad_pick==[1,2,3,5,6,7,8],mad_pick)
# print(rmsd_pick==[1,2,3,5,6,7,8],rmsd_pick)

result of exp1:
Row 1: MAD = 0.15, RMSD = 0.19
Row 2: MAD = 0.19, RMSD = 0.21
Row 3: MAD = 0.14, RMSD = 0.17
Row 4: MAD = 0.17, RMSD = 0.23
Row 5: MAD = 0.14, RMSD = 0.18
Row 6: MAD = 0.17, RMSD = 0.20
Row 7: MAD = 0.13, RMSD = 0.20
Row 8: MAD = 0.15, RMSD = 0.20
result of exp2:
Row 1: MAD = 0.10, RMSD = 0.16
Row 2: MAD = 0.14, RMSD = 0.15
Row 3: MAD = 0.10, RMSD = 0.13
Row 4: MAD = 0.18, RMSD = 0.22
Row 5: MAD = 0.12, RMSD = 0.14
Row 6: MAD = 0.12, RMSD = 0.15
Row 7: MAD = 0.13, RMSD = 0.17
Row 8: MAD = 0.15, RMSD = 0.18
result of exp3:
Row 1: MAD = 0.14, RMSD = 0.20
Row 2: MAD = 0.23, RMSD = 0.30
Row 3: MAD = 0.11, RMSD = 0.17
Row 4: MAD = 0.21, RMSD = 0.23
Row 5: MAD = 0.21, RMSD = 0.24
Row 6: MAD = 0.19, RMSD = 0.28
Row 7: MAD = 0.18, RMSD = 0.23
Row 8: MAD = 0.15, RMSD = 0.18
result of exp5:
Row 1: MAD = 0.14, RMSD = 0.17
Row 2: MAD = 0.22, RMSD = 0.26
Row 3: MAD = 0.11, RMSD = 0.13
Row 4: MAD = 0.19, RMSD = 0.23
Row 5: MAD = 0.19, RMSD = 0.21
Row 6: MAD = 0.13, RMSD = 0.18
Row 7:

In [19]:
uq_data_h = np.array(UQ_list)
uq_data = uq_data_h


In [20]:
exp_index = 0
uq(uq_data[:, exp_index,0]), uq(uq_data[:, exp_index,1]), uq(uq_data[:, exp_index,2]), uq(uq_data[:, exp_index,3]), uq(uq_data[:, exp_index,4]), uq(uq_data[:, exp_index,5]), uq(uq_data[:, exp_index,6]), uq(uq_data[:, exp_index,7])


((0.21320073032598766, 0.0005890159987371453, 0.024269651805024838),
 (0.18342101107178138, 0.0005357778220817667, 0.02314687499602844),
 (0.18146744677925777, 0.0007634772182706633, 0.02763109151428266),
 (0.2496747521867583, 0.003098192168515343, 0.05566140645470022),
 (0.18779673028235008, 0.00010648544043487173, 0.010319178282928915),
 (0.19457573489252541, 0.0002057255210346829, 0.014343134979309191),
 (0.2632467883567712, 0.0036591756257102585, 0.060491120221981824),
 (0.21956559319004487, 0.002070267012746859, 0.04550018695287811))

In [21]:
exp_index = 1
uq(uq_data[:, exp_index,0]), uq(uq_data[:, exp_index,1]), uq(uq_data[:, exp_index,2]), uq(uq_data[:, exp_index,3]), uq(uq_data[:, exp_index,4]), uq(uq_data[:, exp_index,5]), uq(uq_data[:, exp_index,6]), uq(uq_data[:, exp_index,7])


((0.20473342871620134, 0.0020155945203316596, 0.04489537303923044),
 (0.1499600301006339, 0.0006484768520491939, 0.02546520865905469),
 (0.15713068343486433, 0.0007379940111700888, 0.02716604518825088),
 (0.254878789230895, 0.0031632078491277624, 0.05624240259028558),
 (0.1731528719123265, 0.0009259446083415282, 0.030429337954374365),
 (0.1636004843851769, 0.0008602460082576585, 0.029329950703294038),
 (0.2653319636909809, 0.00533813184062412, 0.07306252008125726),
 (0.2053572516766538, 0.003223542031420634, 0.056776245309289645))

In [22]:
exp_index = 2
uq(uq_data[:, exp_index,0]), uq(uq_data[:, exp_index,1]), uq(uq_data[:, exp_index,2]), uq(uq_data[:, exp_index,3]), uq(uq_data[:, exp_index,4]), uq(uq_data[:, exp_index,5]), uq(uq_data[:, exp_index,6]), uq(uq_data[:, exp_index,7])


((0.2034490465635664, 0.0013806190403334519, 0.037156682310634945),
 (0.23982957065332927, 0.001336145977082837, 0.036553330588098766),
 (0.1884997742053302, 0.0001039234101878274, 0.010194283211085878),
 (0.22763406171890774, 0.001446450468769484, 0.03803222934261787),
 (0.18564931510502453, 0.0010514673872804008, 0.03242633786415606),
 (0.23761076211272822, 0.002253001378526512, 0.04746579166648874),
 (0.21033656968694592, 0.0016482161653421934, 0.040598228598575495),
 (0.18181633222505522, 0.000251713099827683, 0.015865468786886917))

In [23]:
exp_index = 3
uq(uq_data[:, exp_index,0]), uq(uq_data[:, exp_index,1]), uq(uq_data[:, exp_index,2]), uq(uq_data[:, exp_index,3]), uq(uq_data[:, exp_index,4]), uq(uq_data[:, exp_index,5]), uq(uq_data[:, exp_index,6]), uq(uq_data[:, exp_index,7])


((0.19806856461445083, 0.000709399911083264, 0.02663456234074936),
 (0.19451155102995138, 0.0016576616164137542, 0.040714390777877967),
 (0.1689807640036775, 0.0011001907740845632, 0.0331691238064041),
 (0.24825356599955767, 0.0013021069047518823, 0.036084718438029725),
 (0.1651974748737159, 0.0014431196157341669, 0.03798841423031721),
 (0.19275492561655178, 0.00027686341567517204, 0.01663921319279166),
 (0.24880057303408515, 0.003134399221114216, 0.05598570550697933),
 (0.18110143997347058, 0.0007862218820211817, 0.02803964839332301))

In [24]:
exp_index = 4
uq(uq_data[:, exp_index,0]), uq(uq_data[:, exp_index,1]), uq(uq_data[:, exp_index,2]), uq(uq_data[:, exp_index,3]), uq(uq_data[:, exp_index,4]), uq(uq_data[:, exp_index,5]), uq(uq_data[:, exp_index,6]), uq(uq_data[:, exp_index,7])


((0.24326094213377852, 0.0001383572923726449, 0.01176253766721471),
 (0.27415494827475223, 0.0007860503320650412, 0.02803658916603518),
 (0.22549744595075136, 0.0014115274087161409, 0.03757029955584785),
 (0.2524614638394046, 0.0011139767380882824, 0.03337629005878698),
 (0.25286783607152974, 0.001836290157595392, 0.04285195628667835),
 (0.23368920580941208, 0.0015995986282065194, 0.039994982537894914),
 (0.2563075067674576, 0.0007713795615520026, 0.027773720700547174),
 (0.24809958316222272, 0.0014040084163187846, 0.03747010029768782))

In [25]:
exp_index = 5
uq(uq_data[:, exp_index,0]), uq(uq_data[:, exp_index,1]), uq(uq_data[:, exp_index,2]), uq(uq_data[:, exp_index,3]), uq(uq_data[:, exp_index,4]), uq(uq_data[:, exp_index,5]), uq(uq_data[:, exp_index,6]), uq(uq_data[:, exp_index,7])


((0.19728522819349453, 0.0024554205736524244, 0.04955220049253539),
 (0.2555269267604495, 0.0015781457758240677, 0.039725882945808366),
 (0.17309978670430545, 0.00010705115391712926, 0.010346552755247965),
 (0.19851049695300702, 0.0009099287526580875, 0.030165025321688153),
 (0.22206350068560393, 0.0005373517573658296, 0.023180848935399876),
 (0.238688808860032, 0.003004651781025338, 0.05481470405854015),
 (0.1726617265477144, 0.001305580544828904, 0.03613281811357791),
 (0.20032773183460012, 0.00014009205800526717, 0.011836049087650286))

In [26]:
exp_index = 6
uq(uq_data[:, exp_index,0]), uq(uq_data[:, exp_index,1]), uq(uq_data[:, exp_index,2]), uq(uq_data[:, exp_index,3]), uq(uq_data[:, exp_index,4]), uq(uq_data[:, exp_index,5]), uq(uq_data[:, exp_index,6]), uq(uq_data[:, exp_index,7])


((0.23471151068351298, 0.0007512439516133096, 0.02740882981109025),
 (0.2003167459263507, 0.0018968315508057273, 0.04355262966579317),
 (0.18200267550695076, 0.0011766698472603944, 0.034302621579995814),
 (0.2675303747728557, 0.00038138099054916836, 0.0195289782259382),
 (0.20200503941802456, 0.0005712197496581561, 0.023900203966873507),
 (0.2002347628750088, 0.0006854373720789874, 0.02618085888734339),
 (0.2540958965244811, 0.0012309343935987041, 0.03508467462580641),
 (0.20373528989611253, 0.000889235596082705, 0.029820053589534427))

In [28]:
uq_data_all=uq_data_c+uq_data_h*10
uq_data = uq_data_all


In [29]:
exp_index = 0
uq(uq_data[:, exp_index,0]), uq(uq_data[:, exp_index,1]), uq(uq_data[:, exp_index,2]), uq(uq_data[:, exp_index,3]), uq(uq_data[:, exp_index,4]), uq(uq_data[:, exp_index,5]), uq(uq_data[:, exp_index,6]), uq(uq_data[:, exp_index,7])


((4.530901407637797, 0.14416249037830048, 0.3796873587285999),
 (5.720867807584465, 0.22657671591308617, 0.47600075200895026),
 (4.910762456228886, 0.22799988271008798, 0.47749333263417193),
 (5.491976193481953, 0.3384414866345172, 0.5817572402940226),
 (5.322451413796717, 0.19954917495859173, 0.4467092734190681),
 (5.8707976313444314, 0.10487511238895664, 0.32384427181742254),
 (5.973751007266355, 0.42018103643991545, 0.6482137274386555),
 (6.400806753690482, 0.3688735091119987, 0.607349577353931))

In [30]:
exp_index = 1
uq(uq_data[:, exp_index,0]), uq(uq_data[:, exp_index,1]), uq(uq_data[:, exp_index,2]), uq(uq_data[:, exp_index,3]), uq(uq_data[:, exp_index,4]), uq(uq_data[:, exp_index,5]), uq(uq_data[:, exp_index,6]), uq(uq_data[:, exp_index,7])


((6.212478980913108, 0.280789933500118, 0.5298961535056826),
 (4.019339146424912, 0.23058384481574262, 0.4801914668293707),
 (5.092322800474991, 0.28795735918453025, 0.5366165848951467),
 (6.651123618869724, 0.208670371869552, 0.45680452260190246),
 (4.959835709506395, 0.44742385458103273, 0.6688974918334144),
 (4.6658533226661385, 0.04399823462887621, 0.209757561553514),
 (6.101730598699096, 0.6300379479897591, 0.7937492979459945),
 (5.3376765756144495, 0.4429353924018309, 0.6655339152904463))

In [31]:
exp_index = 2
uq(uq_data[:, exp_index,0]), uq(uq_data[:, exp_index,1]), uq(uq_data[:, exp_index,2]), uq(uq_data[:, exp_index,3]), uq(uq_data[:, exp_index,4]), uq(uq_data[:, exp_index,5]), uq(uq_data[:, exp_index,6]), uq(uq_data[:, exp_index,7])


((4.383659056682515, 0.326228129127378, 0.5711638373771382),
 (5.391116789975687, 0.18197361764138803, 0.4265836584321861),
 (4.290403412889426, 0.09674854705540598, 0.3110442847174755),
 (5.029498011889691, 0.052083335966793785, 0.22821773806344192),
 (5.106792872860465, 0.05134267020091682, 0.22658921024823053),
 (5.4986428486014685, 0.2611338258371915, 0.5110125495887469),
 (4.706882249521923, 0.1560015162092452, 0.3949702725639554),
 (5.327554134996498, 0.034839901419165355, 0.18665449745228577))

In [32]:
exp_index = 3
uq(uq_data[:, exp_index,0]), uq(uq_data[:, exp_index,1]), uq(uq_data[:, exp_index,2]), uq(uq_data[:, exp_index,3]), uq(uq_data[:, exp_index,4]), uq(uq_data[:, exp_index,5]), uq(uq_data[:, exp_index,6]), uq(uq_data[:, exp_index,7])


((4.93946190369193, 0.06724767436428442, 0.2593215655596048),
 (4.281072481880983, 0.049296814152560244, 0.22202885882821685),
 (4.460948187518428, 0.20472271363064035, 0.4524629417208003),
 (5.191497882668035, 0.1463626197610795, 0.38257367886601856),
 (3.46264344831184, 0.21013383849217698, 0.45840357600282416),
 (4.330554805410824, 0.045990687136938845, 0.21445439407235015),
 (5.199765849049335, 0.4240664738510091, 0.6512038650461229),
 (4.319689468624807, 0.24414665206908603, 0.49411198332876527))

In [33]:
exp_index = 4
uq(uq_data[:, exp_index,0]), uq(uq_data[:, exp_index,1]), uq(uq_data[:, exp_index,2]), uq(uq_data[:, exp_index,3]), uq(uq_data[:, exp_index,4]), uq(uq_data[:, exp_index,5]), uq(uq_data[:, exp_index,6]), uq(uq_data[:, exp_index,7])


((7.211794452052189, 0.1732267797337534, 0.41620521348699296),
 (6.248542931451558, 0.21039010736840663, 0.45868301403955064),
 (6.646253518715009, 0.23695403290335948, 0.486779244528112),
 (6.897253355735605, 0.18046409877181618, 0.42481066226239683),
 (6.7564005594445105, 0.29681574630606417, 0.5448079903104067),
 (5.566940446220487, 0.38942665805878846, 0.6240405900731045),
 (6.40752993863515, 0.11171653023159875, 0.3342402283262725),
 (5.8500559083159445, 0.046291234200967916, 0.21515397788785573))

In [34]:
exp_index = 5
uq(uq_data[:, exp_index,0]), uq(uq_data[:, exp_index,1]), uq(uq_data[:, exp_index,2]), uq(uq_data[:, exp_index,3]), uq(uq_data[:, exp_index,4]), uq(uq_data[:, exp_index,5]), uq(uq_data[:, exp_index,6]), uq(uq_data[:, exp_index,7])


((4.8309670372548865, 0.45573165703260177, 0.6750790005863031),
 (6.32473737203456, 0.7248099112484125, 0.8513576870202163),
 (4.51133174157607, 0.4393022442191977, 0.6627987961811621),
 (4.575601445958884, 0.09655947969669545, 0.3107402125517318),
 (5.594539508191816, 0.1371587811967214, 0.37034953921494407),
 (5.428156522829949, 0.5328861636350571, 0.7299905229761939),
 (4.88407558208673, 0.14687818455915913, 0.3832468976510562),
 (5.591573530590405, 0.26364755593556566, 0.513466216936972))

In [35]:
exp_index = 6
uq(uq_data[:, exp_index,0]), uq(uq_data[:, exp_index,1]), uq(uq_data[:, exp_index,2]), uq(uq_data[:, exp_index,3]), uq(uq_data[:, exp_index,4]), uq(uq_data[:, exp_index,5]), uq(uq_data[:, exp_index,6]), uq(uq_data[:, exp_index,7])


((5.837154908082753, 0.16588063144101278, 0.4072844601025342),
 (6.2134213490592725, 0.43268102471524383, 0.6577849380422478),
 (5.903113913852115, 0.1705528337070714, 0.41298042775302485),
 (5.483547947995947, 0.12480451398203958, 0.35327682344308914),
 (5.134541507664927, 0.1691046686993294, 0.41122338053584623),
 (5.394863585162978, 0.4098465522259212, 0.6401925899492442),
 (6.18934624473638, 0.2934206927883022, 0.5416832033470321),
 (5.318776344052113, 0.4401488800252614, 0.6634371711211706))